# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [1]:
import numpy as np
import ufl

from petsc4py import PETSc
from mpi4py import MPI
from dolfinx import fem
from dolfinx.mesh import CellType, MeshTags, create_box, locate_entities_boundary
L = 20.0
mesh = create_box(MPI.COMM_WORLD,[[0.0,0.0,0.0], [L, 1, 1]], [20, 5, 5], CellType.hexahedron)
V = fem.VectorFunctionSpace(mesh, ("CG", 2))

We create two python functions for determining the facets to apply boundary conditions to

In [2]:
def left(x):
    return np.isclose(x[0], 0)

def right(x):
    return np.isclose(x[0], L)

left_facets = locate_entities_boundary(mesh, mesh.topology.dim - 1, left)
right_facets = locate_entities_boundary(mesh, mesh.topology.dim - 1, right)

Next, we create a  marker based on these two functions

In [3]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full(len(left_facets), 1, dtype=np.int32), np.full(len(right_facets), 2, dtype=np.int32)])
sorted_facets = np.argsort(marked_facets)
facet_tag = MeshTags(mesh, mesh.topology.dim-1, marked_facets[sorted_facets], marked_values[sorted_facets])

We then create a function for supplying the boundary condition on the left side, which is fixed.

In [4]:
u_bc = np.array((0,) * mesh.geometry.dim, dtype=PETSc.ScalarType)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [5]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.indices[facet_tag.values==1])
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`). 

In [6]:
B = fem.Constant(mesh, PETSc.ScalarType((0, 0, 0)))
T = fem.Constant(mesh, PETSc.ScalarType((0, 0, 0)))

Define the test and solution functions on the space $V$

In [7]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [8]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J  = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [9]:
# Elasticity parameters
E = PETSc.ScalarType(1.0e4)
nu = PETSc.ScalarType(0.3)
mu = fem.Constant(mesh, E/(2*(1 + nu)))
lmbda = fem.Constant(mesh, E*nu/((1 + nu)*(1 - 2*nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [10]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [11]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P)*dx - ufl.inner(v, B)*dx - ufl.inner(v, T)*ds(2) 

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [12]:
problem = fem.NonlinearProblem(F, u, bcs)

and then create and customize the Newton solver

In [13]:
from dolfinx.nls import NewtonSolver
solver = NewtonSolver(MPI.COMM_WORLD, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"

We create a function to plot the solution at each time step.

In [14]:
import pyvista
pyvista.set_jupyter_backend("ipygany")

from dolfinx.plot import create_vtk_topology
topology, cell_types = create_vtk_topology(mesh, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, mesh.geometry.x)

def plot_function(t, uh):
    """
    Create a figure of the concentration uh warped visualized in 3D at timet step t.
    """
    p = pyvista.Plotter()
   
    # Update point values on pyvista grid

    topology, cell_types = create_vtk_topology(V)
     # We create a geometry for our modified mesh using the dof coordinates
    geometry = V.tabulate_dof_coordinates()
    # As we are dealing with a vector field, we reshape the underlying dof array to accommedate for the three dimensional space
    num_dofs = V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts
    values = np.zeros((num_dofs, 3), dtype=np.float64)
    values[:, :mesh.geometry.dim] = uh.x.array.real.reshape(num_dofs, V.dofmap.index_map_bs)

    # Create grid defined by the function space for visualization
    function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
    var_name = f"u({t})"
    function_grid[var_name] = values
    function_grid.set_active_vectors(var_name)
    # Warp mesh by deformation
    warped = function_grid.warp_by_vector(var_name, factor=1)
    
    # Add mesh to plotter and visualize
    actor = p.add_mesh(warped)
    p.show_axes()
    if not pyvista.OFF_SCREEN:
       p.show()
    else:
        figure_as_array = p.screenshot(f"diffusion_{t:.2f}.png")
        # Clear plotter for next plot
        p.remove_actor(actor)

plot_function(0, u)

  warnings.warn("Plotting of higher order functions is experimental.")



AppLayout(children=(VBox(children=(HTML(value='<h3>u(0)</h3>'), Dropdown(description='Colormap:', options={'Br…

Finally, we solve the problem over several time steps, updating the y-component of the traction

In [15]:
from dolfinx.log import set_log_level, LogLevel
set_log_level(LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert(converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    plot_function(n, u)

2022-01-07 20:29:20.660 (   1.136s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:29:21.501 (   1.976s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:29:21.963 (   2.438s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 2: r (abs) = 22.2455 (tol = 1e-08) r (rel) = 0.134278(tol = 1e-08)
2022-01-07 20:29:22.178 (   2.654s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:29:22.640 (   3.116s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 3: r (abs) = 2.43261 (tol = 1e-08) r (rel) = 0.0146837(tol = 1e-08)
2022-01-07 20:29:22.859 (   3.335s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:29:23.306 (   3.782s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 

Time step 1, Number of iterations 8, Load [ 0.   0.  -1.5]


2022-01-07 20:29:25.361 (   5.836s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:29:25.741 (   6.216s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 8: r (abs) = 2.62988e-11 (tol = 1e-08) r (rel) = 1.58745e-13(tol = 1e-08)
2022-01-07 20:29:25.741 (   6.216s) [main            ]       NewtonSolver.cpp:250   INFO| Newton solver finished in 8 iterations and 8 linear solver iterations.


AppLayout(children=(VBox(children=(HTML(value='<h3>u(1)</h3>'), Dropdown(description='Colormap:', options={'Br…

2022-01-07 20:29:26.010 (   6.486s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:29:26.603 (   7.079s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:29:26.985 (   7.460s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 2: r (abs) = 17.3254 (tol = 1e-08) r (rel) = 0.117842(tol = 1e-08)
2022-01-07 20:29:27.174 (   7.650s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:29:27.556 (   8.032s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 3: r (abs) = 5.14882 (tol = 1e-08) r (rel) = 0.0350207(tol = 1e-08)
2022-01-07 20:29:27.744 (   8.220s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:29:28.125 (   8.600s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 

...

AppLayout(children=(VBox(children=(HTML(value='<h3>u(8)</h3>'), Dropdown(description='Colormap:', options={'Br…

2022-01-07 20:29:58.381 (  38.857s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:29:59.001 (  39.476s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:29:59.399 (  39.874s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 2: r (abs) = 0.898789 (tol = 1e-08) r (rel) = 0.0309666(tol = 1e-08)
2022-01-07 20:29:59.591 (  40.067s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.


Time step 9, Number of iterations 6, Load [  0.    0.  -13.5]


2022-01-07 20:29:59.974 (  40.449s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 3: r (abs) = 1.38354 (tol = 1e-08) r (rel) = 0.0476679(tol = 1e-08)
2022-01-07 20:30:00.163 (  40.638s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:30:00.542 (  41.018s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 4: r (abs) = 0.00185096 (tol = 1e-08) r (rel) = 6.37724e-05(tol = 1e-08)
2022-01-07 20:30:00.728 (  41.204s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:30:01.113 (  41.589s) [main            ]       NewtonSolver.cpp:34    INFO| Newton iteration 5: r (abs) = 7.87183e-06 (tol = 1e-08) r (rel) = 2.71213e-07(tol = 1e-08)
2022-01-07 20:30:01.301 (  41.777s) [main            ]              petsc.cpp:769   INFO| PETSc Krylov solver starting to solve system.
2022-01-07 20:30:01.682 (  42.158s) [main            ]  

AppLayout(children=(VBox(children=(HTML(value='<h3>u(9)</h3>'), Dropdown(description='Colormap:', options={'Br…